<a href="https://colab.research.google.com/github/ykitaguchi77/GAN/blob/master/APDrawingGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Few shot GAN adaptation**

Implement: http://cedro3.com/ai/few-shot/ <br>
Original: https://github.com/utkarshojha/few-shot-gan-adaptation


#**Set up**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# github からコードを取得
! git clone https://github.com/cedro3/few-shot-gan-adaptation.git
%cd few-shot-gan-adaptation
 
# ライブラリーのインストール
! pip install torch==1.7.0+cu101 torchvision==0.8.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html
! pip install pytorch-fid==0.1.1
! pip install visdom==0.1.8.9
! pip install lpips==0.1.3
! pip install scipy==1.1.0
 
# ninja システムのインストール
! wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
! sudo unzip ninja-linux.zip -d /usr/local/bin/
! sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force
 
# 学習済みパラメータのダウンロード
import gdown
gdown.download('https://drive.google.com/u/0/uc?id=1TQ_6x74RPQf03mSjtqUijM4MZEMyn7HI', 'source_ffhq.pt', quiet=False)
gdown.download('https://drive.google.com/u/0/uc?id=1aZjnGe2E6RCL7Y0Is9klzyYDl7nQzS7Z', 'ffhq_sketches.pt', quiet=False)
gdown.download('https://drive.google.com/u/0/uc?id=18OQmu0HYVr0gEF88BNMY6W519JiW0PZZ', 'ffhq_caricatures.pt', quiet=False)
 
# フォルダーの作成
import os
os.makedirs('traversals', exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Cloning into 'few-shot-gan-adaptation'...
remote: Enumerating objects: 205, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 205 (delta 20), reused 20 (delta 20), pack-reused 180
Receiving objects: 100% (205/205), 97.85 MiB | 42.82 MiB/s, done.
Resolving deltas: 100% (63/63), done.
/content/few-shot-gan-adaptation/few-shot-gan-adaptation
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Lo

Downloading...
From: https://drive.google.com/u/0/uc?id=1TQ_6x74RPQf03mSjtqUijM4MZEMyn7HI
To: /content/few-shot-gan-adaptation/few-shot-gan-adaptation/source_ffhq.pt
100%|██████████| 827M/827M [00:02<00:00, 282MB/s]

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/u/0/uc?id=1aZjnGe2E6RCL7Y0Is9klzyYDl7nQzS7Z 



Access denied with the following error:
Access denied with the following error:



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/u/0/uc?id=18OQmu0HYVr0gEF88BNMY6W519JiW0PZZ 



#**Make "Sketch" illustrations**

In [ ]:
!wget -P /content/few-shot-gan-adaptation/test_sample https://drive.google.com/file/d/1hC5mYgaamaLU9Jyi-WKtK1MoLhyc26kD/view?usp=sharing

--2022-06-16 14:46:32--  https://drive.google.com/file/d/1hC5mYgaamaLU9Jyi-WKtK1MoLhyc26kD/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 142.251.33.206, 2607:f8b0:4004:837::200e
Connecting to drive.google.com (drive.google.com)|142.251.33.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘/content/few-shot-gan-adaptation/test_sample/view?usp=sharing’

view?usp=sharing        [ <=>                ]  64.79K  --.-KB/s    in 0.02s   

2022-06-16 14:46:32 (3.48 MB/s) - ‘/content/few-shot-gan-adaptation/test_sample/view?usp=sharing’ saved [66345]



In [ ]:
# Targetから画像生成
! python generate_img.py --ckpt_target /content/drive/MyDrive/Deep_learning/Few_shot_GAN_adaptation/ffhq_sketches.pt --load_noise noise.pt
 
from IPython.display import Image, display_png
display_png(Image('./test_sample/sample.png'))

#**Photo to sketch_movie**

In [ ]:
# source & target の画像補完
! python generate_img.py --ckpt_source /content/drive/MyDrive/Deep_learning/Few_shot_GAN_adaptation/source_ffhq.pt --ckpt_target /content/drive/MyDrive/Deep_learning/Few_shot_GAN_adaptation/ffhq_sketches.pt --mode interpolate --load_noise noise.pt
 
# output.mp4 ファイル削除
if os.path.exists('./output.mp4'):
   os.remove('./output.mp4')
 
# traversals フォルダーの静止画から動画を作成
! ffmpeg -r 15 -i traversals/sample%3d.png -vcodec libx264 -pix_fmt yuv420p output.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
# 動画再生
from IPython.display import HTML
from base64 import b64encode
 
mp4 = open('./output.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="50%" height="50%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

#**Create anime images**

In [ ]:
! python generate_img.py --ckpt_target /content/drive/MyDrive/Deep_learning/Few_shot_GAN_adaptation/ffhq_caricatures.pt --load_noise noise.pt
 
from IPython.display import Image, display_png
display_png(Image('./test_sample/sample.png'))

#**Photo to anime_movie**

In [ ]:
! python generate_img.py --ckpt_source source_ffhq.pt --ckpt_target ffhq_caricatures.pt\
                           --mode interpolate --load_noise noise.pt
 
# output.mp4 ファイル削除
if os.path.exists('./output.mp4'):
   os.remove('./output.mp4')
 
# traversals フォルダーの静止画から動画を作成
! ffmpeg -r 15 -i traversals/sample%3d.png\
               -vcodec libx264 -pix_fmt yuv420p output.mp4

In [ ]:
# 動画再生
from IPython.display import HTML
from base64 import b64encode
 
mp4 = open('./output.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="50%" height="50%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")